# Stripe Migration Analysis

This notebook performs a comprehensive analysis of migrating customers to new pricing plans.


## 1. Setup and Imports


In [230]:
import pandas as pd
from pathlib import Path


# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)


## 2. Configuration and Constants


In [231]:
# Brand Plans (IN_HOUSE customers)
BRAND_PLANS = {
    "eur": {
        "month": {
            "starter": {
                "price": 89,
                "credits": 3560,
                "price_per_credit": 89 / 3560,
            },
            "pro": {
                "price": 199,
                "credits": 14925,
                "price_per_credit": 199 / 14925,
            },
            "enterprise": {
                "price": 499,
                "credits": 49900,
                "price_per_credit": 499 / 49900,
            },
        },
        "year": {
            "starter": {
                "price": 89 * 10 / 12,
                "credits": 3560,
                "price_per_credit": (89 * 10 / 12) / 3560,
            },
            "pro": {
                "price": 199 * 10 / 12,
                "credits": 14925,
                "price_per_credit": (199 * 10 / 12) / 14925,
            },
            "enterprise": {
                "price": 499 * 10 / 12,
                "credits": 49900,
                "price_per_credit": (499 * 10 / 12) / 49900,
            },
        },
    },
    "usd": {
        "month": {
            "starter": {
                "price": 89,
                "credits": 3560,
                "price_per_credit": 89 / 3560,
            },
            "pro": {
                "price": 199,
                "credits": 14925,
                "price_per_credit": 199 / 14925,
            },
            "enterprise": {
                "price": 499,
                "credits": 49900,
                "price_per_credit": 499 / 49900,
            },
        },
        "year": {
            "starter": {
                "price": 89 * 10 / 12,
                "credits": 3560,
                "price_per_credit": (89 * 10 / 12) / 3560,
            },
            "pro": {
                "price": 199 * 10 / 12,
                "credits": 14925,
                "price_per_credit": (199 * 10 / 12) / 14925,
            },
            "enterprise": {
                "price": 499 * 10 / 12,
                "credits": 49900,
                "price_per_credit": (499 * 10 / 12) / 49900,
            },
        },
    },
}

# Agency Plans
AGENCY_PLANS = {
    "eur": {
        "month": {
            "intro": {
                "price": 89,
                "credits": 2250,
                "price_per_credit": 89 / 2250,
            },
            "growth": {
                "price": 199,
                "credits": 12935,
                "price_per_credit": 199 / 12935,
            },
            "scale": {
                "price": 499,
                "credits": 37425,
                "price_per_credit": 499 / 37425,
            },
        },
        "year": {
            "intro": {
                "price": 89 * 10 / 12,
                "credits": 2250,
                "price_per_credit": (89 * 10 / 12) / 2250,
            },
            "growth": {
                "price": 199 * 10 / 12,
                "credits": 12935,
                "price_per_credit": (199 * 10 / 12) / 12935,
            },
            "scale": {
                "price": 499 * 10 / 12,
                "credits": 37425,
                "price_per_credit": (499 * 10 / 12) / 37425,
            },
        },
    },
    "usd": {
        "month": {
            "intro": {
                "price": 89,
                "credits": 2250,
                "price_per_credit": 89 / 2250,
            },
            "growth": {
                "price": 199,
                "credits": 12935,
                "price_per_credit": 199 / 12935,
            },
            "scale": {
                "price": 499,
                "credits": 37425,
                "price_per_credit": 499 / 37425,
            },
        },
        "year": {
            "intro": {
                "price": 89 * 10 / 12,
                "credits": 2250,
                "price_per_credit": (89 * 10 / 12) / 2250,
            },
            "growth": {
                "price": 199 * 10 / 12,
                "credits": 12935,
                "price_per_credit": (199 * 10 / 12) / 12935,
            },
            "scale": {
                "price": 499 * 10 / 12,
                "credits": 37425,
                "price_per_credit": (499 * 10 / 12) / 37425,
            },
        },
    },
}

# Model pricing (credits per prompt)
MODEL_ID_PRICE_MAP = {
    "gpt-4o": 30,
    "chatgpt": 30,
    "sonar": 30,
    "llama-3-3-70b-instruct": 15,
    "google-ai-overview": 30,
    "gpt-4o-search": 30,
    "gemini-1-5-flash": 30,
    "deepseek-r1": 30,
    "google-ai-mode": 30,
    "gpt-3-5-turbo": 30,
    "claude-sonnet-4": 60,
    "claude-3-5-haiku": 60,
    "gemini-2-5-flash": 60,
    "grok-2-1212": 60,
}


## 3. Define Models

In [232]:
from typing import List, Literal, Optional

from pydantic import BaseModel, ConfigDict
from pydantic.alias_generators import to_camel


class CamelCaseModel(BaseModel):
    """Base model for camelCase to snake_case conversion"""

    model_config = ConfigDict(
        alias_generator=to_camel,
        populate_by_name=True,
    )


class Organization(CamelCaseModel):
    id: str
    company_id: str
    model_ids: List[str]
    prompt_limit: int
    prompts_count: int
    chat_interval_in_hours: int


class Company(CamelCaseModel):
    id: str
    name: str
    type: Literal["IN_HOUSE", "AGENCY", "PARTNER"]
    domain: Optional[str] = None
    stripe_customer_id: str
    stripe_subscription_id: str


class Discount(CamelCaseModel):
    id: str
    percent_off: Optional[int] = None
    amount_off: Optional[int] = None
    duration: Literal["forever", "once", "repeating"]
    duration_in_months: Optional[int] = None


class Subscription(CamelCaseModel):
    id: str
    currency: Literal["eur", "usd"]
    customer: str
    discounts: List[Discount]


class SubscriptionItem(BaseModel):
    """Flattened subscription item model"""
    customer_id: str
    subscription_id: str
    product_id: str
    unit_amount: int
    quantity: int
    interval: Literal["month", "year"]
    interval_count: int
    currency: Literal["eur", "usd"]
    discounts: List[Discount]


class Product(CamelCaseModel):
    id: str
    name: str
    prompt_limit: Optional[int] = None
    type: Optional[Literal["WORKSPACE", "MODELS", "PROMPTS"]] = None


class Coupon(CamelCaseModel):
    id: str
    percent_off: Optional[int] = None
    amount_off: Optional[int] = None
    currency: Optional[Literal["eur", "usd"]] = None
    duration: Literal["forever", "once", "repeating"]
    duration_in_months: Optional[int] = None


class MigrationOutput(BaseModel):
    company_name: str
    company_domain: Optional[str]
    company_type: Literal["IN_HOUSE", "AGENCY", "PARTNER"]
    orgs_count: int
    orgs_count_hf: int
    current_mrr: int
    current_arr: int
    interval: str
    discount: int
    discounts: str
    prompt_usage: int
    prompt_capacity: int
    credits_usage: int
    credits_capacity: int
    plan_name: str
    mrr: int
    mrr_change: int
    arr_change: int
    extra_credits_purchased: int
    surplus_credits: int


print("✓ Models defined")


✓ Models defined


## 4. Load Data

In [233]:
def load_json(file_path: Path):
    """Load JSON and replace NaN with None."""
    df_raw = pd.read_json(file_path)
    records = df_raw.replace({float("nan"): None}).to_dict("records")
    return records

# Define data paths
base_path = Path.cwd().parent.parent
data_path = base_path / "data"

print(f"Loading data from: {data_path}")

# Load all data files
print("Loading source data...")
companies_raw = load_json(data_path / "processed_companies.json")
orgs_raw = load_json(data_path / "processed_organizations.json")
subs_raw = load_json(data_path / "stripe_subscriptions.json")
products_raw = load_json(data_path / "stripe_products.json")

print(f"✓ Loaded {len(companies_raw)} companies")
print(f"✓ Loaded {len(orgs_raw)} organizations")
print(f"✓ Loaded {len(subs_raw)} subscription items")
print(f"✓ Loaded {len(products_raw)} products")


Loading data from: /Users/matevz/dev/peec-ai/stripe-migration-analysis/data
Loading source data...
✓ Loaded 12786 companies
✓ Loaded 3944 organizations
✓ Loaded 1368 subscription items
✓ Loaded 56 products


## 5. Filter and Valdiate Data

In [234]:
# Filter companies
companies_filtered = [
    c
    for c in companies_raw
    if c["stripeSubscriptionId"]
    and c["stripeCustomerId"]
    and c["stripeSubscriptionStatus"] == "active"
]
print(f"Filtered companies: {len(companies_filtered)} (removed {len(companies_raw) - len(companies_filtered)})")

# Flatten and map products
product_mapped = [
    {
        "id": p["id"],
        "name": p["name"],
        "prompt_limit": int(p["metadata"].get("promptLimit"))
        if p["metadata"] and p["metadata"].get("type") == "WORKSPACE"
        else None,
        "type": p["metadata"].get("type")
        if p["metadata"] and p["metadata"].get("type")
        else None,
    }
    for p in products_raw
    if p["active"]
]
print(f"Filtered products: {len(product_mapped)} (removed {len(products_raw) - len(product_mapped)})")

# Map subscriptions
subscriptions_mapped = []
for sub in subs_raw:
    discounts = []
    for discount in sub.get("discounts", []):
        coupon = discount.get("coupon", {})
        discounts.append({
            "id": discount["id"],
            "percent_off": coupon.get("percent_off"),
            "amount_off": coupon.get("amount_off"),
            "duration": coupon.get("duration"),
            "duration_in_months": coupon.get("duration_in_months"),
        })

    subscriptions_mapped.append({
        "id": sub["id"],
        "currency": sub["currency"],
        "customer": sub["customer"],
        "discounts": discounts,
    })


# Flatten subscription items from nested structure
subscription_items_flat = []
for sub in subs_raw:
    for item in sub.get("items", {}).get("data", []):
        price = item.get("price", {})
        recurring = price.get("recurring", {})
        discounts = []
        for discount in item.get("discounts", []):
            coupon = discount.get("coupon", {})
            discounts.append({
                "id": discount["id"],
                "percent_off": coupon.get("percent_off"),
                "amount_off": coupon.get("amount_off"),
                "duration": coupon.get("duration"),
                "duration_in_months": coupon.get("duration_in_months"),
            })
        
        subscription_items_flat.append({
            "customer_id": sub["customer"],
            "subscription_id": sub["id"],
            "product_id": price.get("product"),
            "unit_amount": price.get("unit_amount"),
            "quantity": item.get("quantity", 1),  # Default to 1 if missing
            "interval": recurring.get("interval"),
            "interval_count": recurring.get("interval_count"),
            "currency": sub["currency"],
            "discounts": discounts,
        })

print(f"Flattened subscription items: {len(subscription_items_flat)}")

# Validate with Pydantic models
companies = [Company.model_validate(c) for c in companies_filtered]
orgs = [Organization.model_validate(o) for o in orgs_raw]
products = [Product.model_validate(p) for p in product_mapped]
subscriptions = [Subscription.model_validate(s) for s in subscriptions_mapped]
subscription_items = [SubscriptionItem.model_validate(s) for s in subscription_items_flat]

print(f"✓ Data validated: {len(companies)} companies, {len(orgs)} orgs, {len(products)} products, {len(subscriptions)} subscriptions, {len(subscription_items)} subscription items")


Filtered companies: 1334 (removed 11452)
Filtered products: 48 (removed 8)
Flattened subscription items: 1655
✓ Data validated: 1334 companies, 3944 orgs, 48 products, 1368 subscriptions, 1655 subscription items


## 6. Create DataFrames

In [235]:
# Convert validated models to DataFrames
companies_df = pd.DataFrame([c.model_dump() for c in companies])
orgs_df = pd.DataFrame([o.model_dump() for o in orgs])
subs_df = pd.DataFrame([s.model_dump() for s in subscriptions])
sub_items_df = pd.DataFrame([s.model_dump() for s in subscription_items])
products_df = pd.DataFrame([p.model_dump() for p in products])

print("✓ DataFrames created:")
print(f"  Companies: {companies_df.shape}")
print(f"  Organizations: {orgs_df.shape}")
print(f"  Subscriptions: {subs_df.shape}")
print(f"  Subscription Items: {sub_items_df.shape}")
print(f"  Products: {products_df.shape}")

✓ DataFrames created:
  Companies: (1334, 6)
  Organizations: (3944, 6)
  Subscriptions: (1368, 4)
  Subscription Items: (1655, 9)
  Products: (48, 4)


## 7. Calculate Orgs Credits Usage

In [236]:
def calculate_credits_usage(row: pd.Series) -> int:
    """Calculate required credits based on model usage and run frequency."""
    model_prices = [MODEL_ID_PRICE_MAP.get(mid, 0) for mid in row["model_ids"]]
    return int(sum(model_prices) * row["prompts_count"])

def calculate_credits_capacity(row: pd.Series) -> int:
    """Calculate required credits based on prompt capacity and run frequency."""
    model_prices = [MODEL_ID_PRICE_MAP.get(mid, 0) for mid in row["model_ids"]]
    return int(sum(model_prices) * row["prompt_limit"])

# Calculate credits for each organization
orgs_df["credits_usage"] = orgs_df.apply(calculate_credits_usage, axis=1)
orgs_df["credits_capacity"] = orgs_df.apply(calculate_credits_capacity, axis=1)

print("✓ Calculated credits for organizations")
orgs_df[["id", "company_id", "prompts_count", "prompt_limit", "credits_usage", "credits_capacity"]].head()

✓ Calculated credits for organizations


,id,company_id,prompts_count,prompt_limit,credits_usage,credits_capacity
0,20da1ff7-bed2-40e8-a5c0-cade5250e7ba,co_1fea122e-be87-47f8-b459-4bb426706d35,23,30,2760,3600
1,25f8bb17-0754-4840-ada6-40e7a9345f27,co_f5267b94-1922-4312-8e0d-b3b2b20864fa,21,25,2205,2625
2,28b0f80a-4e7a-4936-97b8-838150c78f70,co_ae8a374a-3893-4c21-857c-1bea3a469807,34,30,3060,2700
3,3d08ee7f-b5bd-4324-8524-f0f97ead5245,co_1f216996-4e82-46a6-9089-97c619ecf16c,10,120,1200,14400
4,4e838a3a-eb44-4378-bda1-c94de3357279,co_66c7c3ff-4038-4dcc-b225-89f7fc5e212a,33,55,6930,11550


## 8. Subscription Data

In [237]:
sub_items_df.head()

,customer_id,subscription_id,product_id,unit_amount,quantity,interval,interval_count,currency,discounts
0,cus_TJxLetjVm2EAZN,sub_1SNWrHKojVEYZPlXWHT4HDpP,prod_S6Vr6mr8BJrWGc,202900,1,year,1,eur,[]
1,cus_TJyLxk98uMqThC,sub_1SNKP0KojVEYZPlXyjCqMNot,prod_SF88MEged9XbuX,59900,1,month,1,eur,[]
2,cus_TJwUAXCMDgad93,sub_1SNIcGKojVEYZPlXcc1HaA4n,prod_SzGUmLVdXEWA8R,100000,1,month,1,eur,[]
3,cus_TJrQO7gNsYGcvQ,sub_1SNEzdKojVEYZPlX29Kdoi0y,prod_S6Vr6mr8BJrWGc,19900,1,month,1,eur,[]
4,cus_TJmmyVqlmcwJiW,sub_1SN9D4KojVEYZPlXKWHHlWDr,prod_RfyjDqozRtX8Lp,6000,3,month,1,eur,[]


In [238]:
# Step 2: Process line item discounts
def apply_item_discounts(row):
    """Apply long-term discounts to a subscription item."""
    unit_amount = row['unit_amount']
    quantity = row['quantity']
    interval_count = row['interval_count']
    
    if unit_amount is None or pd.isna(unit_amount):
        return pd.Series({'total_amount': 0, 'discounted_amount': 0, 'discount_count': 0})
    
    # Multiply unit amount by quantity to get total line item amount
    total_amount = unit_amount * quantity / interval_count
    discounted_amount = total_amount
    discount_count = 0
    
    # Process each discount on this item (discounts are now nested directly)
    for discount in row['discounts']:
        # Extract coupon data directly from the nested discount
        duration = discount.get('duration')
        duration_in_months = discount.get('duration_in_months')
        amount_off = discount.get('amount_off')
        percent_off = discount.get('percent_off')
        
        # Only apply long-term discounts (forever or repeating >= 12 months)
        is_forever = duration == 'forever'
        is_long_term_repeating = (
            duration == 'repeating' and 
            duration_in_months is not None and
            duration_in_months >= 12
        )
        
        if not (is_forever or is_long_term_repeating):
            continue
        
        # Apply amount_off discount (applies to line item total, not per unit)
        if amount_off is not None and not pd.isna(amount_off):
            discounted_amount -= amount_off
            discount_count += 1
        # Apply percent_off discount
        elif percent_off is not None and not pd.isna(percent_off):
            discounted_amount *= (100 - percent_off) / 100
            discount_count += 1
    
    return pd.Series({
        'total_amount': total_amount,
        'discounted_amount': max(0, discounted_amount),
        'discount_count': discount_count
    })

# Apply discounts to each item
sub_items_df[['total_amount', 'discounted_unit_amount', 'item_discount_count']] = sub_items_df.apply(apply_item_discounts, axis=1)

print(f"✓ Calculated discounted amounts for {len(sub_items_df)} subscription items")


sub_items_df.sort_values(by="item_discount_count", ascending=False).head()

✓ Calculated discounted amounts for 1655 subscription items


,customer_id,subscription_id,product_id,unit_amount,quantity,interval,interval_count,currency,discounts,total_amount,discounted_unit_amount,item_discount_count
1636,cus_Rr6wUVkUBGgKhu,sub_1QxOinKojVEYZPlXUC8hkH1J,prod_SKjjWEnxjSeydd,4900,1,month,1,eur,"[{'id': 'di_1RQ4KsKojVEYZPlXSrqSPfLp', 'percen...",4900.0,0.0,1.0
1607,cus_S1KF1sae2h24f3,sub_1R7HatKojVEYZPlXnwcbBpQD,prod_SJLngGx2JOogoI,49900,1,month,1,eur,"[{'id': 'di_1RQC0cKojVEYZPlXHok4hVfP', 'percen...",49900.0,40200.0,1.0
1015,cus_SkHM57oOU0rYHJ,sub_1RompbKojVEYZPlX2mGQiRTO,prod_T1ABr4linMlSmm,1900,1,month,1,eur,"[{'id': 'di_1SIn3CKojVEYZPlXFzUOa81r', 'percen...",1900.0,0.0,1.0
1542,cus_SBWWynndgSlFcq,sub_1RH9TAKojVEYZPlXEclydvvj,prod_S6Vr6mr8BJrWGc,19900,9,month,1,eur,"[{'id': 'di_1RXLdNKojVEYZPlX1o3umdSM', 'percen...",179100.0,161190.0,1.0
1541,cus_SBWWynndgSlFcq,sub_1RH9TAKojVEYZPlXEclydvvj,prod_S6Vq3DJcPoXe3i,8900,6,month,1,eur,"[{'id': 'di_1RXLdNKojVEYZPlXIcNfZTca', 'percen...",53400.0,45390.0,1.0


In [239]:
# Step 3: Aggregate items to subscription level
sub_aggregated = (
    sub_items_df.groupby("subscription_id")
    .agg(
        {
            "total_amount": "sum",
            "discounted_unit_amount": "sum",
            "item_discount_count": "sum",
            "interval": lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],  # Most common interval
            "currency": "first",
        }
    )
    .reset_index()
)

sub_aggregated.rename(
    columns={"discounted_unit_amount": "base_amount"},
    inplace=True,
)

print(f"✓ Aggregated {len(sub_aggregated)} subscriptions")
sub_aggregated.head()

✓ Aggregated 1368 subscriptions


,subscription_id,total_amount,base_amount,item_discount_count,interval,currency
0,sub_1QmeiLKojVEYZPlXi5MWl3yK,26700.0,26700.0,0.0,month,eur
1,sub_1Qn6CJKojVEYZPlXWJbZYtmw,13800.0,13800.0,0.0,month,eur
2,sub_1Qr0qTKojVEYZPlXHvzPi4Za,130000.0,130000.0,0.0,month,eur
3,sub_1Qs6r7KojVEYZPlXmUSaEzTm,35000.0,35000.0,0.0,month,eur
4,sub_1QsFwnKojVEYZPlXWipZ7TE6,18000.0,18000.0,0.0,month,eur


In [240]:
# Step 4: Apply subscription-level discounts
def apply_subscription_discounts(row, subs_df):
    """Apply long-term subscription-level discounts."""
    sub_id = row["subscription_id"]
    amount = row["base_amount"]

    # Get subscription discounts (coupons are now nested directly in subscriptions)
    sub_data = subs_df[subs_df["id"] == sub_id]
    if sub_data.empty:
        return pd.Series({"final_amount": amount, "sub_discount_count": 0})

    discounts_list = sub_data.iloc[0]["discounts"]
    if not discounts_list:
        return pd.Series({"final_amount": amount, "sub_discount_count": 0})

    discounted_amount = amount
    discount_count = 0

    for discount in discounts_list:
        # Extract coupon data directly from the nested discount
        duration = discount.get("duration")
        duration_in_months = discount.get("duration_in_months")
        amount_off = discount.get("amount_off")
        percent_off = discount.get("percent_off")

        # Only apply long-term discounts (forever or repeating >= 12 months)
        is_forever = duration == "forever"
        is_long_term_repeating = (
            duration == "repeating"
            and duration_in_months is not None
            and duration_in_months >= 12
        )

        if not (is_forever or is_long_term_repeating):
            continue

        # Apply amount_off discount
        if amount_off is not None and not pd.isna(amount_off):
            discounted_amount -= amount_off
            discount_count += 1
        # Apply percent_off discount
        elif percent_off is not None and not pd.isna(percent_off):
            discounted_amount *= (100 - percent_off) / 100
            discount_count += 1

    return pd.Series(
        {
            "final_amount": max(0, discounted_amount),
            "sub_discount_count": discount_count,
        }
    )


sub_aggregated[["final_amount", "sub_discount_count"]] = sub_aggregated.apply(
    lambda row: apply_subscription_discounts(row, subs_df), axis=1
)

# Calculate total discount count
sub_aggregated["discount_count"] = (
    sub_aggregated["item_discount_count"] + sub_aggregated["sub_discount_count"]
)

sub_aggregated = sub_aggregated.drop(columns=["total_amount"])

print("✓ Applied subscription-level discounts")
sub_aggregated.head()

✓ Applied subscription-level discounts


,subscription_id,base_amount,item_discount_count,interval,currency,final_amount,sub_discount_count,discount_count
0,sub_1QmeiLKojVEYZPlXi5MWl3yK,26700.0,0.0,month,eur,17800.0,1.0,1.0
1,sub_1Qn6CJKojVEYZPlXWJbZYtmw,13800.0,0.0,month,eur,13800.0,0.0,0.0
2,sub_1Qr0qTKojVEYZPlXHvzPi4Za,130000.0,0.0,month,eur,130000.0,0.0,0.0
3,sub_1Qs6r7KojVEYZPlXmUSaEzTm,35000.0,0.0,month,eur,35000.0,0.0,0.0
4,sub_1QsFwnKojVEYZPlXWipZ7TE6,18000.0,0.0,month,eur,18000.0,0.0,0.0


In [ ]:
# Step 5: Calculate MRR/ARR based on interval
def calculate_mrr_arr(row):
    """Calculate MRR and ARR based on interval."""
    amount = row['final_amount']
    interval = row['interval']
    
    if interval == 'month':
        return pd.Series({'mrr': amount / 100, 'arr': None})  # Convert cents to dollars/euros
    elif interval == 'year':
        arr = amount / 100
        mrr = arr / 12
        return pd.Series({'mrr': mrr, 'arr': arr})
    else:
        raise ValueError(f"Invalid interval: {interval}")

sub_aggregated[['mrr', 'arr']] = sub_aggregated.apply(calculate_mrr_arr, axis=1)

print(f"Total MRR: {(sub_aggregated['mrr'].sum()).round(0).astype(int) * 12:,}")
print("✓ Calculated MRR/ARR")

Total MRR: 3,472,524
✓ Calculated MRR/ARR


KeyError: 'company_id'

In [242]:
# Step 6: Join with companies data
# First, merge subscription data with subs_df to get customer_id
subscription_data = sub_aggregated.merge(
    subs_df[['id', 'customer']],
    left_on='subscription_id',
    right_on='id',
    how='left'
).drop(columns=['id'])

subscription_data.rename(columns={'customer': 'customer_id'}, inplace=True)

# Now merge with companies to get company_id
subscription_data = subscription_data.merge(
    companies_df,
    left_on='subscription_id',
    right_on='stripe_subscription_id',
    how='left'
)

subscription_data.rename(columns={'id': 'company_id'}, inplace=True)


# Step 7: Create final DataFrame with required columns
subscription_data_df = subscription_data[[
    'name',
    'company_id',
    'customer_id', 
    'subscription_id',
    'mrr',
    'arr',
    'discount_count',
    'currency',
    'interval',
]].copy()

print(f"\n✓ Created subscription_data_df with {len(subscription_data_df)} subscriptions")
print(f"  Monthly subscriptions: {subscription_data_df['arr'].isna().sum()}")
print(f"  Yearly subscriptions: {subscription_data_df['arr'].notna().sum()}")
print(f"  Subscriptions with discounts: {(subscription_data_df['discount_count'] > 0).sum()}")
print("\nSample data:")

subscription_data.head(10)


✓ Created subscription_data_df with 1371 subscriptions
  Monthly subscriptions: 1326
  Yearly subscriptions: 45
  Subscriptions with discounts: 27

Sample data:


,subscription_id,base_amount,item_discount_count,interval,currency,final_amount,sub_discount_count,discount_count,mrr,arr,customer_id,company_id,name,type,domain,stripe_customer_id,stripe_subscription_id
0,sub_1QmeiLKojVEYZPlXi5MWl3yK,26700.0,0.0,month,eur,17800.0,1.0,1.0,178.0,NaN,cus_Rg0jFvpFlajive,co_2ebe593d-1e29-4669-8e0f-9e315cd56f5a,Antler,IN_HOUSE,antler.co,cus_Rg0jFvpFlajive,sub_1QmeiLKojVEYZPlXi5MWl3yK
1,sub_1Qn6CJKojVEYZPlXWJbZYtmw,13800.0,0.0,month,eur,13800.0,0.0,0.0,138.0,NaN,cus_RgT8o2F5OOUaSZ,co_66c7c3ff-4038-4dcc-b225-89f7fc5e212a,Glide,IN_HOUSE,glideapps.com,cus_RgT8o2F5OOUaSZ,sub_1Qn6CJKojVEYZPlXWJbZYtmw
2,sub_1Qr0qTKojVEYZPlXHvzPi4Za,130000.0,0.0,month,eur,130000.0,0.0,0.0,1300.0,NaN,cus_RkVsLVkfHVfbgw,co_3b72236b-8e31-40a4-a1e8-0e764d0a66a7,Peak Ace,AGENCY,peakace.agency,cus_RkVsLVkfHVfbgw,sub_1Qr0qTKojVEYZPlXHvzPi4Za
3,sub_1Qs6r7KojVEYZPlXmUSaEzTm,35000.0,0.0,month,eur,35000.0,0.0,0.0,350.0,NaN,cus_Rle9u38yh4qQwV,co_7dd92ff3-4b2e-4f2d-820e-943aa303f0a5,Something Inc.,AGENCY,somethinginc.com,cus_Rle9u38yh4qQwV,sub_1Qs6r7KojVEYZPlXmUSaEzTm
4,sub_1QsFwnKojVEYZPlXWipZ7TE6,18000.0,0.0,month,eur,18000.0,0.0,0.0,180.0,NaN,cus_RlnXKGF0oxvbL9,co_8a9b51da-b538-4227-a77d-1c5095f454d1,VertoDigital,AGENCY,vertodigital.com,cus_RlnXKGF0oxvbL9,sub_1QsFwnKojVEYZPlXWipZ7TE6
5,sub_1QsJ8mKojVEYZPlXn7AcJGYQ,168000.0,0.0,month,eur,168000.0,0.0,0.0,1680.0,NaN,cus_RlqqHruUNFA2Xa,co_75a298ac-a7ee-411a-a001-3eca23dcf1c8,Suchhelden,AGENCY,suchhelden.de,cus_RlqqHruUNFA2Xa,sub_1QsJ8mKojVEYZPlXn7AcJGYQ
6,sub_1QuWuQKojVEYZPlXr9wtEj5I,19900.0,0.0,month,eur,19900.0,0.0,0.0,199.0,NaN,cus_Ro9CwwoUwQ4J1I,co_af273fff-420d-4707-a44f-f06dac7589d5,Superside,IN_HOUSE,superside.com,cus_Ro9CwwoUwQ4J1I,sub_1QuWuQKojVEYZPlXr9wtEj5I
7,sub_1QvyDQKojVEYZPlXiKerghsd,21000.0,0.0,month,eur,21000.0,0.0,0.0,210.0,NaN,cus_RpdUTheiW3FXhN,co_1fea122e-be87-47f8-b459-4bb426706d35,Flow Agency,AGENCY,flow-agency.com,cus_RpdUTheiW3FXhN,sub_1QvyDQKojVEYZPlXiKerghsd
8,sub_1QwLeAKojVEYZPlXe0EwbFeN,33000.0,0.0,month,eur,33000.0,0.0,0.0,330.0,NaN,cus_Rq1hmmxhI6OuWE,co_653ed519-88e1-4c7d-9866-93f16feb8f9a,saas.group,AGENCY,addsearch.com,cus_Rq1hmmxhI6OuWE,sub_1QwLeAKojVEYZPlXe0EwbFeN
9,sub_1QxNxGKojVEYZPlXy9fhtN82,19900.0,0.0,month,eur,19900.0,0.0,0.0,199.0,NaN,cus_Rr69XZNAWsWkRq,co_a5b9f4f0-d872-4fed-a217-d41a3c7284a9,Skribble,IN_HOUSE,skribble.com,cus_Rr69XZNAWsWkRq,sub_1QxNxGKojVEYZPlXy9fhtN82


## Purchased workspace prompts

In [79]:
items_with_product = sub_items_df.merge(
    products_df[["id", "name", "prompt_limit", "type"]],
    left_on="product_id",
    right_on="id",
    how="left",
)

# filter workspace items
sub_items_with_product = items_with_product[items_with_product["type"] == "WORKSPACE"]
print(f"Workspace items: {len(sub_items_with_product)}")
sub_items_with_product = sub_items_with_product[sub_items_with_product["unit_amount"] > 0]
print(f"Workspace items with amount > 0: {len(sub_items_with_product)}")

# group by customer_id and sum the prompt_limit and count workspace items
company_stats_stripe = (
    sub_items_with_product.groupby("customer_id")
    .agg({
        "prompt_limit": "sum",
        "quantity": "sum"  # Count of workspace items with unit_amount > 0
    })
    .reset_index()
    .rename(columns={
        "prompt_limit": "purchased_capacity",
        "quantity": "orgs_purchased"
    })
)


company_stats_stripe.sort_values(by="orgs_purchased", ascending=False).head(10)


Workspace items: 1514
Workspace items with amount > 0: 1453


,customer_id,purchased_capacity,orgs_purchased
992,cus_T6Ibs3QB4FA2MN,10.0,150
1291,cus_TFiW0oljAK4r0J,15.0,50
138,cus_SMIS7gO4lnP88j,125.0,40
816,cus_SyUrmlDvfJLlqc,10.0,30
409,cus_SebOt23y9b2U8Y,175.0,30
497,cus_SjAzdi2phnFbYx,30.0,27
4,cus_RlqqHruUNFA2Xa,30.0,27
624,cus_Soty5ZCdLV79mv,50.0,22
513,cus_SjdjN04eJFj1HW,195.0,21
983,cus_T5ZyYnbSzbNC7H,15.0,20


## 9. Capacity Data

In [80]:
company_stats_fs = (
    orgs_df.groupby("company_id")
    .agg(
        orgs_count=("company_id", "size"),  # Count of organizations per company
        prompts_count=("prompts_count", "sum"),
        prompt_limit=("prompt_limit", "sum"),
        credits_usage=("credits_usage", "sum"),
        credits_capacity=("credits_capacity", "sum"),
        orgs_count_hf=("chat_interval_in_hours", lambda x: (x < 24).sum()),
    )
    .reset_index()
)

company_stats_fs.head()


,company_id,orgs_count,prompts_count,prompt_limit,credits_usage,credits_capacity,orgs_count_hf
0,co_004d676c-c61c-4888-bcf8-6ef606a156ed,1,6,25,540,2250,0
1,co_004f67b6-6249-4c2b-9cf7-4c39dfb8b86e,1,5,25,450,2250,0
2,co_0066473d-9106-4e5f-b13c-7b756c207675,7,197,505,17730,45450,0
3,co_00a903f4-7115-4770-bde3-6e8eb9982243,1,25,25,2250,2250,0
4,co_00e9c907-6659-4829-9a93-558923266790,1,100,100,9000,9000,0


In [81]:
main = companies_df.merge(
    company_stats_stripe,
    left_on="stripe_customer_id",
    right_on="customer_id",
    how="left",
)

main = main.merge(
    company_stats_fs,
    left_on="id",
    right_on="company_id",
    how="left",
)

main = main.merge(
    subscription_data_df,
    left_on="id",
    right_on="company_id",
    how="left",
)

main.head()

,id,name,type,domain,stripe_customer_id,stripe_subscription_id,customer_id_x,purchased_capacity,orgs_purchased,company_id_x,orgs_count,prompts_count,prompt_limit,credits_usage,credits_capacity,orgs_count_hf,company_id_y,customer_id_y,subscription_id,mrr,arr,discount_count,currency,interval
0,co_0066473d-9106-4e5f-b13c-7b756c207675,Flying Cat,AGENCY,flyingcatmarketing.com,cus_T6T5U4MOGZ7ntZ,sub_1SDr10KojVEYZPlXy4X5spim,cus_T6T5U4MOGZ7ntZ,300.0,1.0,co_0066473d-9106-4e5f-b13c-7b756c207675,7.0,197.0,505.0,17730.0,45450.0,0.0,co_0066473d-9106-4e5f-b13c-7b756c207675,cus_T6T5U4MOGZ7ntZ,sub_1SDr10KojVEYZPlXy4X5spim,499.0,NaN,0.0,eur,month
1,co_00e9c907-6659-4829-9a93-558923266790,Wickey,IN_HOUSE,wickey.de,cus_T40lDnXfwWLTJe,sub_1SD0bmKojVEYZPlXA2382RPf,cus_T40lDnXfwWLTJe,100.0,1.0,co_00e9c907-6659-4829-9a93-558923266790,1.0,100.0,100.0,9000.0,9000.0,0.0,co_00e9c907-6659-4829-9a93-558923266790,cus_T40lDnXfwWLTJe,sub_1SD0bmKojVEYZPlXA2382RPf,199.0,NaN,0.0,eur,month
2,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,Dot Dash,AGENCY,dotdash.io,cus_T2bSdUiWSeuKVX,sub_1SGhkrKojVEYZPlXgxccGh4K,cus_T2bSdUiWSeuKVX,25.0,1.0,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,1.0,20.0,25.0,1800.0,2250.0,0.0,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,cus_T2bSdUiWSeuKVX,sub_1SGhkrKojVEYZPlXgxccGh4K,89.0,NaN,0.0,usd,month
3,co_018bcf89-5317-4104-88df-9f6e77a52276,TrueClicks,IN_HOUSE,trueclicks.com,cus_SytMawkXSQzCwo,sub_1S2vbmKojVEYZPlX79pt2DcB,cus_SytMawkXSQzCwo,25.0,1.0,co_018bcf89-5317-4104-88df-9f6e77a52276,1.0,9.0,25.0,810.0,2250.0,0.0,co_018bcf89-5317-4104-88df-9f6e77a52276,cus_SytMawkXSQzCwo,sub_1S2vbmKojVEYZPlX79pt2DcB,89.0,NaN,0.0,eur,month
4,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,CommsCo,AGENCY,thecommsco.com,cus_SlmRXxPfcvT5vJ,sub_1RqEu2KojVEYZPlX9mMIMKnT,cus_SlmRXxPfcvT5vJ,25.0,4.0,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,4.0,76.0,100.0,6840.0,9000.0,0.0,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,cus_SlmRXxPfcvT5vJ,sub_1RqEu2KojVEYZPlX9mMIMKnT,280.0,NaN,0.0,eur,month


## 10. Migration Analysis


In [82]:
# Step 2: Calculate baseline_credits_needed
# Use the higher of what they're using vs what they have configured
main['baseline_credits_needed'] = main[['credits_usage', 'credits_capacity']].max(axis=1)

print("✓ Calculated baseline_credits_needed")
print(f"  Average baseline credits: {main['baseline_credits_needed'].mean():.0f}")
main[['name', 'credits_usage', 'credits_capacity', 'baseline_credits_needed']].head()


✓ Calculated baseline_credits_needed
  Average baseline credits: 11435


,name,credits_usage,credits_capacity,baseline_credits_needed
0,Flying Cat,17730.0,45450.0,45450.0
1,Wickey,9000.0,9000.0,9000.0
2,Dot Dash,1800.0,2250.0,2250.0
3,TrueClicks,810.0,2250.0,2250.0
4,CommsCo,6840.0,9000.0,9000.0


In [83]:
# Step 3: Filter migration_df (exclude 100% discounts and missing data)
# Calculate effective discount percentage
main['effective_discount_pct'] = 0
has_mrr = main['mrr'].notna()
has_arr = main['arr'].notna()

# For monthly subscriptions
monthly_mask = has_mrr & main['mrr'].notna()
# Calculate what they should pay for baseline credits
# We'll use this later, but for now just filter

# Filter out companies with issues
migration_df = main[
    # Must have subscription data
    (has_mrr | has_arr) &
    # Must have credit data
    main['baseline_credits_needed'].notna() &
    # Exclude 100% discount/free accounts (paying less than 1 EUR/USD)
    ((main['mrr'].fillna(0) > 1) | (main['arr'].fillna(0) > 12))
].copy()

# print names and domains of excluded companies
excluded_companies = main[~main['id'].isin(migration_df['id'])]

print("✓ Filtered migration_df")
print(f"  Total companies in main: {len(main)}")
print(f"  Companies for migration: {len(migration_df)}")
print(f"  Excluded (100% discount or missing data): {len(main) - len(migration_df)}")
print(f"  By type: {migration_df['type'].value_counts().to_dict()}")

migration_df.head()


✓ Filtered migration_df
  Total companies in main: 1325
  Companies for migration: 1272
  Excluded (100% discount or missing data): 53
  By type: {'IN_HOUSE': 732, 'AGENCY': 540}


,id,name,type,domain,stripe_customer_id,stripe_subscription_id,customer_id_x,purchased_capacity,orgs_purchased,company_id_x,orgs_count,prompts_count,prompt_limit,credits_usage,credits_capacity,orgs_count_hf,company_id_y,customer_id_y,subscription_id,mrr,arr,discount_count,currency,interval,baseline_credits_needed,effective_discount_pct
0,co_0066473d-9106-4e5f-b13c-7b756c207675,Flying Cat,AGENCY,flyingcatmarketing.com,cus_T6T5U4MOGZ7ntZ,sub_1SDr10KojVEYZPlXy4X5spim,cus_T6T5U4MOGZ7ntZ,300.0,1.0,co_0066473d-9106-4e5f-b13c-7b756c207675,7.0,197.0,505.0,17730.0,45450.0,0.0,co_0066473d-9106-4e5f-b13c-7b756c207675,cus_T6T5U4MOGZ7ntZ,sub_1SDr10KojVEYZPlXy4X5spim,499.0,NaN,0.0,eur,month,45450.0,0
1,co_00e9c907-6659-4829-9a93-558923266790,Wickey,IN_HOUSE,wickey.de,cus_T40lDnXfwWLTJe,sub_1SD0bmKojVEYZPlXA2382RPf,cus_T40lDnXfwWLTJe,100.0,1.0,co_00e9c907-6659-4829-9a93-558923266790,1.0,100.0,100.0,9000.0,9000.0,0.0,co_00e9c907-6659-4829-9a93-558923266790,cus_T40lDnXfwWLTJe,sub_1SD0bmKojVEYZPlXA2382RPf,199.0,NaN,0.0,eur,month,9000.0,0
2,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,Dot Dash,AGENCY,dotdash.io,cus_T2bSdUiWSeuKVX,sub_1SGhkrKojVEYZPlXgxccGh4K,cus_T2bSdUiWSeuKVX,25.0,1.0,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,1.0,20.0,25.0,1800.0,2250.0,0.0,co_01766115-a8ce-40f5-8dc4-b391fcee3db0,cus_T2bSdUiWSeuKVX,sub_1SGhkrKojVEYZPlXgxccGh4K,89.0,NaN,0.0,usd,month,2250.0,0
3,co_018bcf89-5317-4104-88df-9f6e77a52276,TrueClicks,IN_HOUSE,trueclicks.com,cus_SytMawkXSQzCwo,sub_1S2vbmKojVEYZPlX79pt2DcB,cus_SytMawkXSQzCwo,25.0,1.0,co_018bcf89-5317-4104-88df-9f6e77a52276,1.0,9.0,25.0,810.0,2250.0,0.0,co_018bcf89-5317-4104-88df-9f6e77a52276,cus_SytMawkXSQzCwo,sub_1S2vbmKojVEYZPlX79pt2DcB,89.0,NaN,0.0,eur,month,2250.0,0
4,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,CommsCo,AGENCY,thecommsco.com,cus_SlmRXxPfcvT5vJ,sub_1RqEu2KojVEYZPlX9mMIMKnT,cus_SlmRXxPfcvT5vJ,25.0,4.0,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,4.0,76.0,100.0,6840.0,9000.0,0.0,co_01dbefdf-03bd-4788-90c3-8aeb11c359f7,cus_SlmRXxPfcvT5vJ,sub_1RqEu2KojVEYZPlX9mMIMKnT,280.0,NaN,0.0,eur,month,9000.0,0


In [84]:
# Step 4: Plan selection helper functions
def select_plan_for_company(row):
    """
    Select the most expensive plan that fits within the company's current MRR/ARR.
    Returns: plan_name, plan_credits, plan_price, credit_price
    """
    company_type = row["type"]
    currency = row["currency"]
    interval = row["interval"]

    # Determine which plan set to use
    if company_type == "IN_HOUSE":
        plans = BRAND_PLANS
    elif company_type == "AGENCY":
        plans = AGENCY_PLANS
    else:  # PARTNER
        plans = BRAND_PLANS  # Default to brand plans

    plans = plans[currency][interval]

    current_payment = row["mrr"]

    # Find all plans that fit within budget
    affordable_plans = []
    for plan_name, plan_data in plans.items():
        plan_price = plan_data["price"]

        if plan_price <= current_payment:
            affordable_plans.append(
                {
                    "name": plan_name,
                    "price": plan_price,
                    "credits": plan_data["credits"],
                    "price_per_credit": plan_data["price_per_credit"],
                }
            )

    # If no affordable plans, use the cheapest one (will need discount)
    if not affordable_plans:
        cheapest_plan_name = min(
            plans.keys(),
            key=lambda x: plans[x][interval]["price"]
            if interval in plans[x]
            else float("inf"),
        )
        cheapest_plan = plans[cheapest_plan_name]
        return pd.Series(
            {
                "new_plan": cheapest_plan_name,
                "plan_credits": cheapest_plan["credits"],
                "plan_price": cheapest_plan["price"],
                "credit_price": cheapest_plan["price_per_credit"],
            }
        )

    # Select the most expensive affordable plan
    selected_plan = max(affordable_plans, key=lambda x: x["price"])

    return pd.Series(
        {
            "new_plan": selected_plan["name"],
            "plan_credits": selected_plan["credits"],
            "plan_price": selected_plan["price"],
            "credit_price": selected_plan["price_per_credit"],
        }
    )


# Apply plan selection
migration_df[["new_plan", "plan_credits", "plan_price", "credit_price"]] = (
    migration_df.apply(select_plan_for_company, axis=1)
)

print("✓ Selected plans for companies")
print("\nPlan distribution:")
print(migration_df["new_plan"].value_counts())
migration_df[["name", "type", "interval", "mrr", "arr", "new_plan", "plan_price"]].head(
    10
)


✓ Selected plans for companies

Plan distribution:
new_plan
starter       489
intro         265
pro           220
growth        206
scale          69
enterprise     23
Name: count, dtype: int64


,name,type,interval,mrr,arr,new_plan,plan_price
0,Flying Cat,AGENCY,month,499.0,NaN,scale,499.0
1,Wickey,IN_HOUSE,month,199.0,NaN,pro,199.0
2,Dot Dash,AGENCY,month,89.0,NaN,intro,89.0
3,TrueClicks,IN_HOUSE,month,89.0,NaN,starter,89.0
4,CommsCo,AGENCY,month,280.0,NaN,growth,199.0
5,Gear4music,IN_HOUSE,month,199.0,NaN,pro,199.0
6,Betmode,IN_HOUSE,month,199.0,NaN,pro,199.0
7,RivalMind,AGENCY,month,199.0,NaN,growth,199.0
8,Harper James,IN_HOUSE,month,499.0,NaN,enterprise,499.0
9,addmustard,AGENCY,month,199.0,NaN,growth,199.0


In [85]:
migration_df.loc[(migration_df['mrr'].isna()) & (migration_df['arr'].isna())]


,id,name,type,domain,stripe_customer_id,stripe_subscription_id,customer_id_x,purchased_capacity,orgs_purchased,company_id_x,orgs_count,prompts_count,prompt_limit,credits_usage,credits_capacity,orgs_count_hf,company_id_y,customer_id_y,subscription_id,mrr,arr,discount_count,currency,interval,baseline_credits_needed,effective_discount_pct,new_plan,plan_credits,plan_price,credit_price


In [86]:
# Step 5: Calculate purchased_credits and extra_credits_granted
def calculate_extra_credits(row):
    """
    Calculate purchased credits (they're paying for) vs granted credits (we give free).
    """
    baseline_needed = row['baseline_credits_needed']
    plan_credits = row['plan_credits']
    plan_price = row['plan_price']
    credit_price = row['credit_price']
    
    # Get current payment
    current_payment = row['mrr']
    
    # Handle missing data
    if pd.isna(baseline_needed) or pd.isna(plan_credits) or pd.isna(current_payment):
        raise ValueError(f"Company {row['name']} has missing data")
    
    # Calculate credit gap
    credits_gap = baseline_needed - plan_credits
    
    # Case C: Plan has enough credits
    if credits_gap <= 0:
        return pd.Series({
            'purchased_credits': 0,
            'extra_credits_granted': 0,
            'extra_credits_granted_10': 0
        })
    
    # Case A: Paying enough for extra credits
    available_for_credits = current_payment - plan_price
    if available_for_credits > 0:
        # They can afford to buy credits
        affordable_credits = available_for_credits / credit_price if credit_price > 0 else 0
        purchased = min(credits_gap, affordable_credits)
        
        # If they still need more after purchasing, grant the rest
        remaining_gap = credits_gap - purchased
        granted = max(0, remaining_gap)
        granted_10 = granted * 1.10
        
        return pd.Series({
            'purchased_credits': purchased,
            'extra_credits_granted': granted,
            'extra_credits_granted_10': granted_10
        })
    
    # Case B: Need credits but not paying enough
    else:
        return pd.Series({
            'purchased_credits': 0,
            'extra_credits_granted': credits_gap,
            'extra_credits_granted_10': credits_gap * 1.10
        })

# Apply credit calculations
migration_df[['purchased_credits', 'extra_credits_granted', 'extra_credits_granted_10']] = \
    migration_df.apply(calculate_extra_credits, axis=1)

print("✓ Calculated extra credits")
print(f"  Companies purchasing credits: {(migration_df['purchased_credits'] > 0).sum()}")
print(f"  Companies needing granted credits: {(migration_df['extra_credits_granted'] > 0).sum()}")

# migration_df.drop(columns=["id", "customer_id_x", "company_id_x", "company_id_y", "customer_id_y", ]).to_csv("migration_df.csv", index=False)

migration_df[['name', 'baseline_credits_needed', 'plan_credits', 'purchased_credits', 'extra_credits_granted']].head(10)


✓ Calculated extra credits
  Companies purchasing credits: 182
  Companies needing granted credits: 113


,name,baseline_credits_needed,plan_credits,purchased_credits,extra_credits_granted
0,Flying Cat,45450.0,37425,0.0,8025.0
1,Wickey,9000.0,14925,0.0,0.0
2,Dot Dash,2250.0,2250,0.0,0.0
3,TrueClicks,2250.0,3560,0.0,0.0
4,CommsCo,9000.0,12935,0.0,0.0
5,Gear4music,9000.0,14925,0.0,0.0
6,Betmode,9000.0,14925,0.0,0.0
7,RivalMind,15750.0,12935,0.0,2815.0
8,Harper James,27000.0,49900,0.0,0.0
9,addmustard,2250.0,12935,0.0,0.0


In [87]:
# Step 6: Calculate discounts for underpaying companies
def calculate_discount(row):
    """
    Calculate discount needed for companies paying less than the cheapest plan.
    """
    company_type = row['type']
    currency = row["currency"]
    interval = row['interval']
    
    # Determine which plan set to use
    if company_type == 'IN_HOUSE':
        plans = BRAND_PLANS
        cheapest_plan_name = 'starter'
    elif company_type == 'AGENCY':
        plans = AGENCY_PLANS
        cheapest_plan_name = 'intro'
    else:  # PARTNER
        plans = BRAND_PLANS
        cheapest_plan_name = 'starter'

    plans = plans[currency][interval]
    
    # Get current payment
    current_payment = row['mrr']
    
    # Handle missing data
    if pd.isna(current_payment):
        raise ValueError(f"Company {row['name']} has no payment data")
    
    # Get cheapest plan price
    cheapest_price = plans[cheapest_plan_name]['price']
    if pd.isna(cheapest_price):
        raise ValueError(f"Company {row['name']} has no cheapest plan price")
    
    # Calculate discount if underpaying
    if current_payment < cheapest_price:
        discount_amount = cheapest_price - current_payment
        discount_pct = (discount_amount / cheapest_price) * 100
        return pd.Series({
            'discount_amount': discount_amount,
            'discount_pct': discount_pct
        })
    else:
        return pd.Series({
            'discount_amount': 0,
            'discount_pct': 0
        })

# Apply discount calculations
migration_df[['discount_amount', 'discount_pct']] = \
    migration_df.apply(calculate_discount, axis=1)

print("✓ Calculated discounts")
print(f"  Companies needing discount: {(migration_df['discount_pct'] > 0).sum()}")
print(f"  Average discount %: {migration_df[migration_df['discount_pct'] > 0]['discount_pct'].mean():.1f}%")

migration_df[migration_df['discount_pct'] > 0][['name', 'mrr', 'arr', 'plan_price', 'discount_amount', 'discount_pct']].head(10)


✓ Calculated discounts
  Companies needing discount: 1
  Average discount %: 14.6%


,name,mrr,arr,plan_price,discount_amount,discount_pct
595,EmberTribe,76.0,NaN,89.0,13.0,14.606742


In [88]:
# Step 7: Calculate total_available_credits
migration_df['total_available_credits'] = (
    migration_df['plan_credits'] + 
    migration_df['purchased_credits'] + 
    migration_df['extra_credits_granted']
)

print("✓ Calculated total_available_credits")
print(f"  Average total credits: {migration_df['total_available_credits'].mean():.0f}")


✓ Calculated total_available_credits
  Average total credits: 13730


In [89]:
# Step 8: Calculate unused_credits
# Total available minus actual usage
migration_df['unused_credits'] = migration_df['total_available_credits'] - migration_df['credits_usage']


# Ensure non-negative
migration_df['unused_credits'] = migration_df['unused_credits'].clip(lower=0)

# print("✓ Calculated unused_credits")
print(f"  Average unused credits: {migration_df['unused_credits'].mean():.0f}")
print(f"  Companies with unused credits: {(migration_df['unused_credits'] > 0).sum()}")

migration_df[['name', 'total_available_credits', 'credits_usage', 'unused_credits']].head(10)


  Average unused credits: 6338
  Companies with unused credits: 1181


,name,total_available_credits,credits_usage,unused_credits
0,Flying Cat,45450.0,17730.0,27720.0
1,Wickey,14925.0,9000.0,5925.0
2,Dot Dash,2250.0,1800.0,450.0
3,TrueClicks,3560.0,810.0,2750.0
4,CommsCo,12935.0,6840.0,6095.0
5,Gear4music,14925.0,9000.0,5925.0
6,Betmode,14925.0,2790.0,12135.0
7,RivalMind,15750.0,15120.0,630.0
8,Harper James,49900.0,23850.0,26050.0
9,addmustard,12935.0,2160.0,10775.0


In [90]:
# Step 9: Calculate unused_purchased_credits
# How many of the PAID extra credits they're not using
# Formula: max(0, purchased_credits - (credits_usage - plan_credits))
migration_df['unused_purchased_credits'] = (
    migration_df['purchased_credits'] - 
    (migration_df['credits_usage'] - migration_df['plan_credits'])
).clip(lower=0)

print("✓ Calculated unused_purchased_credits")
print(f"  Companies with unused purchased credits: {(migration_df['unused_purchased_credits'] > 0).sum()}")
print(f"  Average unused purchased credits: {migration_df[migration_df['unused_purchased_credits'] > 0]['unused_purchased_credits'].mean():.0f}")

migration_df[migration_df['unused_purchased_credits'] > 0][['name', 'purchased_credits', 'credits_usage', 'plan_credits', 'unused_purchased_credits']].head(10)


✓ Calculated unused_purchased_credits
  Companies with unused purchased credits: 1133
  Average unused purchased credits: 5423


,name,purchased_credits,credits_usage,plan_credits,unused_purchased_credits
0,Flying Cat,0.0,17730.0,37425,19695.0
1,Wickey,0.0,9000.0,14925,5925.0
2,Dot Dash,0.0,1800.0,2250,450.0
3,TrueClicks,0.0,810.0,3560,2750.0
4,CommsCo,0.0,6840.0,12935,6095.0
5,Gear4music,0.0,9000.0,14925,5925.0
6,Betmode,0.0,2790.0,14925,12135.0
8,Harper James,0.0,23850.0,49900,26050.0
9,addmustard,0.0,2160.0,12935,10775.0
10,SciChart,0.0,6930.0,14925,7995.0


In [91]:
# Step 10: Calculate MRR at risk (normalized to annual)
# Revenue we'd lose if they drop unused purchased credits
migration_df['arr_at_risk'] = (
    migration_df['unused_purchased_credits'] * 
    migration_df['credit_price'] * 
    12
)

print("✓ Calculated arr_at_risk")
print(f"  Total ARR at risk (annual): ${migration_df['arr_at_risk'].sum():,.2f}")
print(f"  Companies with ARR at risk: {(migration_df['arr_at_risk'] > 0).sum()}")
print(f"  Average ARR at risk: ${migration_df[migration_df['arr_at_risk'] > 0]['arr_at_risk'].mean():,.2f}")

migration_df[migration_df['arr_at_risk'] > 0].sort_values('arr_at_risk', ascending=False)[['name', 'interval', 'unused_purchased_credits', 'credit_price', 'arr_at_risk']].head(10)


✓ Calculated arr_at_risk
  Total ARR at risk (annual): $1,189,951.69
  Companies with ARR at risk: 1133
  Average ARR at risk: $1,050.27


,name,interval,unused_purchased_credits,credit_price,arr_at_risk
594,Propellic,month,36990.0,0.013333,5918.4
1283,Rokk Solutions,month,36165.0,0.013333,5786.4
194,Fotocasa,month,48100.0,0.010000,5772.0
433,rlvnt,month,35175.0,0.013333,5628.0
1069,ai4Suite,month,35175.0,0.013333,5628.0
551,First Page Digital,month,34200.0,0.013333,5472.0
563,Suchhelden,month,33210.0,0.013333,5313.6
1093,Store.is,month,33075.0,0.013333,5292.0
1228,PartnerCentric,month,31035.0,0.013333,4965.6
641,Evergreen Media,month,30960.0,0.013333,4953.6


In [93]:
# Summary Statistics & Validation
print("=" * 80)
print("MIGRATION ANALYSIS SUMMARY")
print("=" * 80)

print("\n📊 OVERALL STATISTICS:")
print(f"  Total companies analyzed: {len(migration_df)}")
print(f"  Total current MRR: ${migration_df['mrr'].fillna(0).sum():,.2f}")
print(f"  Total current ARR: ${(migration_df['mrr'].fillna(0).sum() * 12):,.2f}")

print("\n📋 PLAN DISTRIBUTION:")
plan_counts = migration_df['new_plan'].value_counts()
for plan, count in plan_counts.items():
    pct = (count / len(migration_df)) * 100
    print(f"  {plan}: {count} ({pct:.1f}%)")

print("\n💰 CREDIT ECONOMICS:")
print(f"  Companies purchasing extra credits: {(migration_df['purchased_credits'] > 0).sum()}")
print(f"  Total purchased credits: {migration_df['purchased_credits'].sum():,.0f}")
print(f"  Companies receiving granted credits: {(migration_df['extra_credits_granted'] > 0).sum()}")
print(f"  Total granted credits: {migration_df['extra_credits_granted'].sum():,.0f}")
print(f"  Total granted credits (with 10% buffer): {migration_df['extra_credits_granted_10'].sum():,.0f}")

print("\n🎁 DISCOUNTS:")
print(f"  Companies needing discounts: {(migration_df['discount_pct'] > 0).sum()}")
if (migration_df['discount_pct'] > 0).sum() > 0:
    print(f"  Average discount: {migration_df[migration_df['discount_pct'] > 0]['discount_pct'].mean():.1f}%")
    print(f"  Max discount: {migration_df['discount_pct'].max():.1f}%")

print("\n⚠️  RISK ANALYSIS:")
print(f"  Total unused credits: {migration_df['unused_credits'].sum():,.0f}")
print(f"  Total unused PURCHASED credits: {migration_df['unused_purchased_credits'].sum():,.0f}")
print(f"  Total MRR at risk (annual): ${migration_df['arr_at_risk'].sum():,.2f}")
print(f"  Companies with at-risk revenue: {(migration_df['arr_at_risk'] > 0).sum()}")

print("\n✅ VALIDATION:")
# Check that baseline needs are met
needs_met = migration_df['total_available_credits'] >= migration_df['baseline_credits_needed']
print(f"  Companies with sufficient credits: {needs_met.sum()} / {len(migration_df)}")
if not needs_met.all():
    print(f"  ⚠️  WARNING: {(~needs_met).sum()} companies have insufficient credits!")

print("\n" + "=" * 80)


MIGRATION ANALYSIS SUMMARY

📊 OVERALL STATISTICS:
  Total companies analyzed: 1272
  Total current MRR: $263,504.46
  Total current ARR: $3,162,053.54

📋 PLAN DISTRIBUTION:
  starter: 489 (38.4%)
  intro: 265 (20.8%)
  pro: 220 (17.3%)
  growth: 206 (16.2%)
  scale: 69 (5.4%)
  enterprise: 23 (1.8%)

💰 CREDIT ECONOMICS:
  Companies purchasing extra credits: 182
  Total purchased credits: 2,368,343
  Companies receiving granted credits: 113
  Total granted credits: 3,081,162
  Total granted credits (with 10% buffer): 3,389,279

🎁 DISCOUNTS:
  Companies needing discounts: 1
  Average discount: 14.6%
  Max discount: 14.6%

⚠️  RISK ANALYSIS:
  Total unused credits: 8,061,560
  Total unused PURCHASED credits: 6,144,703
  Total MRR at risk (annual): $1,189,951.69
  Companies with at-risk revenue: 1133

✅ VALIDATION:
  Companies with sufficient credits: 1272 / 1272



In [651]:
# Display final migration dataframe with key columns
key_columns = [
    # Identifiers
    'name', 'type', 'customer_id',
    # Current state
    'currency', 'interval', 'mrr', 'arr', 'discount_count', 'prompts_count', 'prompt_limit',
    # Credit analysis  
    'baseline_credits_needed', 'credits_usage', 'credits_capacity',
    'orgs_count', 'orgs_count_hf', 'orgs_purchased',
    # New plan
    'new_plan', 'plan_credits', 'plan_price',
    # Extra credits
    'purchased_credits', 'extra_credits_granted', 'extra_credits_granted_10',
    # Discounts
    'discount_pct', 'discount_amount',
    # Totals
    'total_available_credits',
    # Risk
    'unused_credits', 'unused_purchased_credits', 'arr_at_risk'
]

# Filter to columns that exist
existing_columns = [col for col in key_columns if col in migration_df.columns]

export_df = migration_df[existing_columns].copy()
export_df['mrr'] = export_df['mrr'].round(0).astype(int)
export_df['discount_count'] = export_df['discount_count'].fillna(0).astype(int)
export_df['baseline_credits_needed'] = export_df['baseline_credits_needed'].round(0).astype(int)
export_df['credits_usage'] = export_df['credits_usage'].round(0).astype(int)
export_df['orgs_count'] = export_df['orgs_count'].round(0).astype(int)
export_df['orgs_count_hf'] = export_df['orgs_count_hf'].round(0).astype(int)
export_df['purchased_credits'] = export_df['purchased_credits'].round(0).astype(int)
export_df['extra_credits_granted'] = export_df['extra_credits_granted'].round(0).astype(int)
export_df['extra_credits_granted_10'] = export_df['extra_credits_granted_10'].round(0).astype(int)
export_df['discount_amount'] = export_df['discount_amount'].round(0).astype(int)
export_df['total_available_credits'] = export_df['total_available_credits'].round(0).astype(int)
export_df['unused_credits'] = export_df['unused_credits'].round(0).astype(int)
export_df['unused_purchased_credits'] = export_df['unused_purchased_credits'].round(0).astype(int)
export_df['arr_at_risk'] = export_df['arr_at_risk'].round(0).astype(int)



export_df.to_csv("migration_df_export.csv", index=False)


print(f"Migration DataFrame with {len(migration_df)} companies and {len(existing_columns)} columns")
print("\nTop 20 companies by current MRR:")
export_df.sort_values('mrr', ascending=False, na_position='last').head(20)


Migration DataFrame with 1242 companies and 27 columns

Top 20 companies by current MRR:


,name,type,currency,interval,mrr,arr,discount_count,prompts_count,prompt_limit,baseline_credits_needed,credits_usage,credits_capacity,orgs_count,orgs_count_hf,orgs_purchased,new_plan,plan_credits,plan_price,purchased_credits,extra_credits_granted,extra_credits_granted_10,discount_pct,discount_amount,total_available_credits,unused_credits,unused_purchased_credits,arr_at_risk
235,UNESCO,IN_HOUSE,usd,month,4970,NaN,0,500.0,500.0,45000,45000,45000.0,1,0,1.0,enterprise,49900,499,0,0,0,0.0,0,49900,4900,4900,588
1025,Butternut Box,IN_HOUSE,eur,month,4320,NaN,0,153.0,200.0,18000,13770,18000.0,1,1,1.0,enterprise,49900,499,0,0,0,0.0,0,49900,36130,36130,4336
82,Advice Interactive,AGENCY,usd,month,2260,NaN,0,1981.0,2050.0,184500,178290,184500.0,3,0,3.0,scale,37425,499,132075,15000,16500,0.0,0,184500,6210,0,0
424,Growth Plays,AGENCY,eur,month,2233,NaN,2,1199.0,1575.0,237000,182670,237000.0,17,2,15.0,scale,37425,499,130069,69506,76457,0.0,0,237000,54330,0,0
597,Propellic,AGENCY,eur,month,2150,NaN,0,505.0,925.0,85500,47520,85500.0,31,0,40.0,scale,37425,499,48075,0,0,0.0,0,85500,37980,37980,6077
1023,Future PLC (Dell),AGENCY,eur,month,2102,NaN,0,1761.0,1900.0,171000,158490,171000.0,6,0,NaN,scale,37425,499,120187,13388,14726,0.0,0,171000,12510,0,0
1146,primelis,AGENCY,eur,month,2000,NaN,0,3195.0,10174.0,1152630,354690,1152630.0,80,0,NaN,scale,37425,499,112575,1002630,1102893,0.0,0,1152630,797940,0,0
1011,Seer Interactive,AGENCY,eur,month,1966,NaN,0,2603.0,5050.0,484500,249270,484500.0,25,0,NaN,scale,37425,499,109995,337080,370788,0.0,0,484500,235230,0,0
978,TIpi Group,AGENCY,eur,month,1900,NaN,0,1573.0,1630.0,146700,141570,146700.0,18,2,18.0,scale,37425,499,105075,4200,4620,0.0,0,146700,5130,930,149
1103,Limitless Agency,AGENCY,eur,month,1809,NaN,0,536.0,874.0,80910,50490,80910.0,29,0,29.0,scale,37425,499,43485,0,0,0.0,0,80910,30420,30420,4867


In [634]:
# Interesting segments for deeper analysis

print("🔍 HIGH-RISK COMPANIES (Top 10 by MRR at risk):")
print("=" * 80)
high_risk = migration_df[migration_df['arr_at_risk'] > 0].sort_values('arr_at_risk', ascending=False).head(10)
print(high_risk[['name', 'mrr', 'unused_purchased_credits', 'arr_at_risk']])

print("\n\n💡 COMPANIES NEEDING LARGE DISCOUNTS (Top 10):")
print("=" * 80)
big_discounts = migration_df[migration_df['discount_pct'] > 0].sort_values('discount_pct', ascending=False).head(10)
if len(big_discounts) > 0:
    print(big_discounts[['name', 'mrr', 'arr', 'plan_price', 'discount_pct', 'discount_amount']])
else:
    print("No companies need discounts!")

print("\n\n🎁 COMPANIES RECEIVING FREE CREDITS (Top 10):")
print("=" * 80)
free_credits = migration_df[migration_df['extra_credits_granted'] > 0].sort_values('extra_credits_granted', ascending=False).head(10)
if len(free_credits) > 0:
    print(free_credits[['name', 'baseline_credits_needed', 'plan_credits', 'extra_credits_granted', 'mrr']])
else:
    print("No companies need free granted credits!")

print("\n\n💰 TOP REVENUE COMPANIES (Top 10 by MRR):")
print("=" * 80)
top_revenue = migration_df.sort_values('mrr', ascending=False, na_position='last').head(10)
print(top_revenue[['name', 'type', 'mrr', 'arr', 'new_plan', 'orgs_count', 'baseline_credits_needed']])


🔍 HIGH-RISK COMPANIES (Top 10 by MRR at risk):
                 name      mrr  unused_purchased_credits  arr_at_risk
101              Mito  1350.00                   89460.0      11928.0
1108         Store.is   765.00                   40500.0       6480.0
597         Propellic  2150.00                   37980.0       6076.8
1281   Rokk Solutions   499.00                   36165.0       5786.4
194          Fotocasa  1194.00                   48100.0       5772.0
432             rlvnt   989.00                   35175.0       5628.0
566        Suchhelden  1560.00                   33930.0       5428.8
636              GAIN   499.00                   31755.0       5080.8
1238   PartnerCentric   578.93                   31035.0       4965.6
646   Evergreen Media  1500.00                   30690.0       4910.4


💡 COMPANIES NEEDING LARGE DISCOUNTS (Top 10):
           name   mrr  arr  plan_price  discount_pct  discount_amount
598  EmberTribe  76.0  NaN          89     14.606742             

## Migration Analysis Complete

The migration analysis has been completed with the following components:

### Methodology
1. **Plan Selection**: Selected the most expensive plan that fits within each company's current MRR/ARR budget
2. **Credit Matching**: Added purchased credits (if they're paying enough) or granted free credits (if they're not) to meet their baseline needs
3. **Discount Calculation**: Calculated required discounts for companies paying less than the cheapest plan
4. **Risk Analysis**: Identified revenue at risk from companies with unused purchased credits

### Key Outputs
- **`migration_df`**: Complete migration plan for each company with:
  - Selected plan and pricing
  - Purchased vs granted credits
  - Discount requirements
  - Risk metrics (unused credits, MRR at risk)

### Next Steps
You can export this data or perform additional analysis:
```python
# Example: Export to CSV
# migration_df.to_csv('../data/migration_plan.csv', index=False)

# Example: Filter specific segments
# high_risk = migration_df[migration_df['mrr_at_risk'] > 100]
# needs_discount = migration_df[migration_df['discount_pct'] > 0]
```
